In [2]:
import pandas as pd
from pandas.io.json import json_normalize
import requests
import os
import json
import sqlite3
from sqlalchemy import create_engine

## FSQ

In [3]:
foursquare_key = os.environ["FOURSQUARE_API_KEY"]
lat = '49.26660189762465'
long = '-123.25003857290854'

headers = {'Accept': 'application/json'}
headers['Authorization'] = foursquare_key

url_append = f'?ll={lat}%2C{long}&radius=5000&categories=13000'

url = "https://api.foursquare.com/v3/places/search"

In [4]:
def fsq_get_venues(lat, long):
    """
    1. Executes get request from FSQ API
    2. Prints status code of get request
    3. Returns normalized DataFrame from JSON file retrieved by the get request
    """
    res = requests.get(url + url_append, headers = headers)
    print('The status code is: ' + str(res.status_code))
    request_json = res.json()

    fsq_data = pd.json_normalize(request_json, record_path = 'results')
    return fsq_data

In [5]:
df = fsq_get_venues(lat, long)

The status code is: 200


In [6]:
parsed_df = df[['name', 'categories', 'location.address']]
parsed_df

,name,categories,location.address
0,Loafe Cafe,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",6163 University Blvd
1,Mercante,"[{'id': 13064, 'name': 'Pizzeria', 'icon': {'p...",6488 University Blvd
2,Great Dane Coffee,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",6011 Walter Gage Rd
3,The Boulevard Coffee Roasting Co,"[{'id': 13035, 'name': 'Coffee Shop', 'icon': ...",5970 University Blvd
4,Koerner's Pub,"[{'id': 10000, 'name': 'Arts and Entertainment...",6371 Crescent Rd
5,Beyond Bread Ltd,"[{'id': 13002, 'name': 'Bakery', 'icon': {'pre...",3686 4th Ave W
6,Baru Cafe,"[{'id': 13068, 'name': 'American Restaurant', ...",2535 Alma St
7,Only U Cafe,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",5737 Dalhousie Rd
8,Breka Downtown Bakery Cafe Inc,"[{'id': 13002, 'name': 'Bakery', 'icon': {'pre...",3750 4th Ave W
9,Rajio Japanese Public House,"[{'id': 13263, 'name': 'Japanese Restaurant', ...",3763W 10th Ave W


In [25]:
# Creating database and inserting fsq table

with sqlite3.connect('data/mini_project_ii.db') as con:
    cur = con.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS fsq_table (id INTEGER PRIMARY KEY AUTOINCREMENT,name TEXT NOT NULL,type TEXT,address TEXT);")
    cur.execute("INSERT INTO fsq_table (name, type, address) VALUES ('Loafe Cafe', 'Café', '6163 University Blvd');")
    cur.execute("INSERT INTO fsq_table (name, type, address) VALUES ('Mercante', 'Pizzeria', '6488 University Blvd');")
    cur.execute("INSERT INTO fsq_table (name, type, address) VALUES ('Great Dane Coffee', 'Café', '6011 Walter Gage Rd');")
    cur.execute("INSERT INTO fsq_table (name, type, address) VALUES ('The Boulevard Coffee Roasting Co', 'Coffee Shop', '5970 University Blvd');")
    cur.execute("INSERT INTO fsq_table (name, type, address) VALUES ('Koerners Pub', 'Arts and Entertainment', '6371 Crescent Rd');")
    cur.execute("INSERT INTO fsq_table (name, type, address) VALUES ('Beyond Bread Ltd', 'Bakery', '3686 4th Ave W');")
    cur.execute("INSERT INTO fsq_table (name, type, address) VALUES ('JJ Bean', 'Coffee Shop', '6005 University Blvd');")
    cur.execute("INSERT INTO fsq_table (name, type, address) VALUES ('Breka Downtown Bakery Cafe Inc', 'Bakery', '3750 4th Ave W');")
    cur.execute("INSERT INTO fsq_table (name, type, address) VALUES ('Rajio Japanese Public House', 'Japanese Restaurant', '3763W 10th Ave W');")
    cur.execute("INSERT INTO fsq_table (name, type, address) VALUES ('Biercraft Wesbrook at UBC', 'Bar', '3340 Shrum Lane');")
    

In [7]:
table = 'fsq_table'

with create_engine('sqlite:///data/mini_project_ii.db').connect() as con:
    fsq_table_df = pd.read_sql_table(table, con)

fsq_table_df

,id,name,type,address
0,1,Loafe Cafe,Café,6163 University Blvd
1,2,Mercante,Pizzeria,6488 University Blvd
2,3,Great Dane Coffee,Café,6011 Walter Gage Rd
3,4,The Boulevard Coffee Roasting Co,Coffee Shop,5970 University Blvd
4,5,Koerners Pub,Arts and Entertainment,6371 Crescent Rd
5,6,Beyond Bread Ltd,Bakery,3686 4th Ave W
6,7,JJ Bean,Coffee Shop,6005 University Blvd
7,8,Breka Downtown Bakery Cafe Inc,Bakery,3750 4th Ave W
8,9,Rajio Japanese Public House,Japanese Restaurant,3763W 10th Ave W
9,10,Biercraft Wesbrook at UBC,Bar,3340 Shrum Lane


## Yelp

In [74]:
yelp_key = os.environ["YELP_API_KEY"]
lat = '49.26660189762465'
long = '-123.25003857290854'

url = "https://api.yelp.com/v3/businesses/search"
headers = {'Authorization': 'Bearer %s' % yelp_key}
params = {'latitude': lat,
          'longitude': long,
          'radius': 5000,
          'categories': 'food'}

In [78]:
def yelp_get_venues(lat, long):
    """
    1. Executes get request from Yelp API
    2. Prints status code of get request
    3. Returns normalized DataFrame from JSON file retrieved by the get request
    """
    res = requests.get(url, params = params, headers = headers)
    print('The status code is: ' + str(res.status_code))
    request_json = res.json()

    yelp_data = pd.json_normalize(request_json, record_path = 'businesses')
    return yelp_data

In [87]:
df = yelp_get_venues(lat, long)

The status code is: 200


In [89]:
parsed_df = df[['name', 'categories', 'location.address1', 'rating']]
parsed_df

,name,categories,location.address1,rating
0,Doughgirls,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",3322 Shrum Lane,4.5
1,Grounds For Coffee,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",2565 Alma Street,4.0
2,The PokéMan,"[{'alias': 'poke', 'title': 'Poke'}, {'alias':...",3742 10th Avenue W,4.5
3,The Boulevard Coffee Roasting,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",5970 University Boulevard,3.5
4,Mercante,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",6488 University Boulevard,3.5
5,Aphrodite's Organic Pie Shop,"[{'alias': 'gluten_free', 'title': 'Gluten-Fre...",3598 W 4th Avenue,3.5
6,Rain or Shine,"[{'alias': 'icecream', 'title': 'Ice Cream & F...",6001 University Boulevard,4.0
7,Caffe W,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3646 W Broadway,4.5
8,Bean Around The World UBC,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",6308 Thunderbird Blvd,4.0
9,Pearl Fever Tea House,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",2182 Western Parkway,3.5


In [82]:
# Creating database and inserting yelp table

with sqlite3.connect('data/mini_project_ii.db') as con:
    cur = con.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS yelp_table (id INTEGER PRIMARY KEY AUTOINCREMENT,name TEXT NOT NULL,type TEXT,address TEXT,rating DECIMAL);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Doughgirls', 'cafes', '3322 Shrum Lane', 4.5);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Grounds For Coffee', 'coffee', '2565 Alma Street', 4.0);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('The PokéMan', 'poke', '3742 10th Avenue W', 4.5);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('The Boulevard Coffee Roasting', 'coffee', '5970 University Boulevard', 3.5);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Mercante', 'pizza', '6488 University Boulevard', 3.5);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Aphrodites Organic Pie Shop', 'gluten_free', '3598 W 4th Avenue', 3.5);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Rain or Shine', 'icecream', '6001 University Boulevard', 4.0);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Caffe W', 'coffee', '3646 W Broadway', 4.5);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Bean Around The World UBC', 'coffee', '6308 Thunderbird Blvd', 4.0);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Pearl Fever Tea House', 'coffee', '2182 Western Parkway', 3.5);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Blue Chip Cookies', 'desserts', '1302-6133 Student Union Boulevard', 4.0);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('JJ Bean', 'coffeeroasteries', '6005 University Boulevard', 4.0);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Perchance', 'coffee', '3363 Dunbar St', 4.5);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Breka Bakery & Cafe', 'bakeries', '3750 W 4th Avenue', 4.0);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Beyond Bread Artisan Bakery', 'bakeries', '3686 W 4th Avenue', 4.0);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('UBC Farm Farmers Market', 'farmersmarket', '3461 Ross Drive', 4.5);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Stongs Market', 'grocery', '4221 Dunbar Street', 4.0);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Bean Around the World Coffees', 'coffee', '4456 10th Avenue W', 4.0);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Mix the Bakery', 'bakeries', '4430 10th Avenue W', 3.5);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Steves Poke Bar', 'poke', '3-5990 University Blvd', 4.5);")
    

In [10]:
table = 'yelp_table'

with create_engine('sqlite:///data/mini_project_ii.db').connect() as con:
    yelp_table_df = pd.read_sql(table, con)

yelp_table_df

,id,name,type,address,rating
0,1,Doughgirls,cafes,3322 Shrum Lane,4.5
1,2,Grounds For Coffee,coffee,2565 Alma Street,4.0
2,3,The PokéMan,poke,3742 10th Avenue W,4.5
3,4,The Boulevard Coffee Roasting,coffee,5970 University Boulevard,3.5
4,5,Mercante,pizza,6488 University Boulevard,3.5
5,6,Aphrodites Organic Pie Shop,gluten_free,3598 W 4th Avenue,3.5
6,7,Rain or Shine,icecream,6001 University Boulevard,4.0
7,8,Caffe W,coffee,3646 W Broadway,4.5
8,9,Bean Around The World UBC,coffee,6308 Thunderbird Blvd,4.0
9,10,Pearl Fever Tea House,coffee,2182 Western Parkway,3.5


## Top 10 POIs based on rating

In [11]:
sql = "SELECT name, rating FROM yelp_table ORDER BY rating DESC LIMIT 10"

with create_engine('sqlite:///data/mini_project_ii.db').connect() as con:
    top_rating_df = pd.read_sql(sql, con)

top_rating_df

,name,rating
0,Doughgirls,4.5
1,The PokéMan,4.5
2,Caffe W,4.5
3,Perchance,4.5
4,UBC Farm Farmers Market,4.5
5,Steves Poke Bar,4.5
6,Grounds For Coffee,4.0
7,Rain or Shine,4.0
8,Bean Around The World UBC,4.0
9,Blue Chip Cookies,4.0
